## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. Please submit only the `*.ipynb` file.

4. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

5. Make sure to use Python 3, not Python 2.

Fill your group name and collaborators below:

In [1]:
GROUPNAME = "Do Not have one"
COLLABORATORS = "Jost Arndt"

---

# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree. Should return `"less"` or `"more"`.
* Test your function on the tuple `('yes', 31, 'good')`,

In [2]:
def decision(x):
    smoke = x[0]
    age = int(x[1])
    diet = x[2]
    
    if smoke == 'yes' and age < 29.5:
        return 'less'
    elif smoke =='yes' and age >= 29.5:
        return 'more'
    elif smoke != 'yes' and diet == 'good':
        return 'less'
    elif smoke != 'yes' and diet != 'good':
        return 'more'

In [3]:
# Test
x = ('yes', 31, 'good')
assert decision(x) == 'more'


## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples. Make sure that the tuples have the same format as above, e.g. `('yes', 31, 'good')`.
* Make extra note of the datatype of each element

In [4]:
def gettest():
    result_list = []
    testdata = open("health-test.txt", "r")
    for line in testdata:
        data_list = line.split(',')
        data_tuple = [data_list[0].rstrip(), float(data_list[1]), data_list[2].rstrip()]
        result_list.append(data_tuple)
    return result_list
        

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and return the ratio of them that are classified as "more".

In [5]:
def evaluate_testset():
    morecount = 0
    lesscount = 0
    testdata = gettest()
    for point in testdata:
        if decision(point) == "more":
            morecount +=1
        elif decision(point) == "less":
            lesscount +=1
    return float(morecount) / float(morecount + lesscount)

## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [6]:
def gettrain():
    pair_list = []
    traindata = open("health-train.txt", "r")
    for line in traindata:
        line_list=line.split(",")
        new_tuple =[line_list[0].rstrip(), float(line_list[1]), line_list[2].rstrip()]
        newpair = [new_tuple, line_list[3].rstrip()]
        pair_list.append(newpair)
    return pair_list

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a, b) = (a[0] != b[0]) + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] != b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes', 31, 'good')`

In [7]:
def neighbor(x, trainset):
    distance = 10000000
    classification = 'less'
    for point in trainset:
        if ( (point[0][0] != x[0] ) + point[0][1] + (point[0][2] != x[2]) ) < distance:
            classification = point[1]
            distance = ((point[0][0] != x[0] ) + point[0][1] + (point[0][2] != x[2]))
    return classification

In [8]:
# Test
x = ('yes', 31, 'good')
assert neighbor(x, gettrain()) == "more"


* Apply both the decision tree and nearest neighbor classifiers on the test set, and return the list of data point(s) for which the two classifiers disagree, and with which probability it happens.
* A data point should look like above, e.g. `('yes', 31, 'good')`.

In [9]:
def compare():
    test_list = gettest()
    Xdisagree = []
    for entry in test_list:
        if neighbor(entry, gettrain()) != decision(entry):
            Xdisagree.append(entry)
    probability = float(len(Xdisagree))/float(len(test_list))
    return Xdisagree, probability

In [10]:
Xdisagree, probability = compare()
assert type(Xdisagree) == list
print(Xdisagree)
assert probability >= 0.0 and probability <= 1.0

[['yes', 21.0, 'poor'], ['no', 50.0, 'good'], ['no', 23.0, 'good'], ['no', 60.0, 'good'], ['no', 18.0, 'good']]


One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [11]:
class NearestMeanClassifier:
    more_counter = 0
    less_counter = 0
    average_more = [0.0,0.0,0.0] 
    average_less = [0.0,0.0,0.0]
    def distance(a, b):
        return (a[0] - b[0]) ** 2 + ((a[1] - b[1]) / 50.0) ** 2 + (a[2] - b[2]) ** 2
    def train(self, dataset):
        for point in dataset:
            if point[1]=="more":
                if point[0][1]=="yes":
                    self.average_more[0]=+1
                if point[0][2]=="poor":
                    self.average_more[2]=+1
                point[0][1]+=1
                self.more_counter += 1
            elif point[1]=="less":
                if point[0][1]=="yes":
                    self.average_less[0]=+1
                if point[0][2]=="poor":
                    self.average_less[2]=+1
                point[0][1]+=1
                self.less_counter += 1
        self.average_less[0] = float(self.average_less[0]/float(self.less_counter))
        self.average_less[1] = float(self.average_less[1]/float(self.less_counter))
        self.average_less[2] = float(self.average_less[2]/float(self.less_counter))
        self.average_more[0] = float(self.average_more[0]/float(self.more_counter))
        self.average_more[1] = float(self.average_more[1]/float(self.more_counter))
        self.average_more[2] = float(self.average_more[2]/float(self.more_counter))
                
    def predict(self, x):
        if x[0]=="yes":
            x[0]=1.0
        elif x[0] == "no":
            x[0]= 0
        if x[2]=="poor":
            x[2]= 1.0
        elif x[2] == "good":
            x[2]=0.0
        if ((self.average_less[0]-x[0])**2 +((self.average_less[1]-x[1])/50.0)**2 + (self.average_less[2]-x[2])**2  < (self.average_more[0]-float(x[0]))**2 + ((self.average_more[1]-float(x[1]))/50.0)**2 + (self.average_more[2]-float(x[2]))**2):
            return "less"
        else:
            return "more"

* Build an object of class `NearestMeanClassifier`, train it on the training data, and return the mean vector for each class. You should return a dictionary with keys `less` and `more`. Each key should correspond to a mean vector.

In [12]:
def build_and_train():
    Objekt = NearestMeanClassifier()
    Objekt.train(gettrain())
    mean_more = Objekt.average_more
    mean_less = Objekt.average_less
build_and_train()

* Predict the test data using the nearest mean classifier and return all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree. You should return a list containing tuples. Each tuple should contain the datapoint and the prediction i.e.

`[(('no', 50, 'good'), 'less'), ... ]`

In [13]:
def predict_test():
    Object = NearestMeanClassifier()
    Object.train(gettrain())
    test = gettest()
    agreed_samples = []
    for point in test:
        if Object.predict(point) == decision(point) and decision(point)== neighbor(point, gettrain()):
            tup = (point, decision(x))
            agreed_samples.append(tup)
    return agreed_samples
predict_test()

[([1.0, 21.0, 1.0], 'more'),
 ([1.0, 45.0, 1.0], 'more'),
 ([0, 15.0, 1.0], 'more')]